# LAB 2:  AutoML Tables Babyweight Training.

**Learning Objectives**

1. Setup Vertex AI
1. Create and import dataset from BigQuery
1. Analyze dataset
1. Train AutoML Tables model
1. Check evaluation metrics
1. Deploy model
1. Make batch predictions
1. Make online predictions


## Introduction 
In this notebook, we will use AutoML Tables to train a model to predict the weight of a baby before it is born.  We will use the Vertex AI UI to create a training dataset from BigQuery and will then train, evaluate, and predict with a Auto ML Tables model.

In this lab, we will setup AutoML Tables, create and import an AutoML Tables dataset from BigQuery, analyze AutoML Tables dataset, train an AutoML Tables model, check evaluation metrics of trained model, deploy trained model, and then finally make both batch and online predictions using the trained model.

Each learning objective will correspond to a series of steps to complete in this student lab notebook.

## Verify tables exist

Run the following cells to verify that we previously created the dataset and data tables. If not, go back to lab [1b_prepare_data_babyweight](../solutions/1b_prepare_data_babyweight.ipynb) to create them.

In [ ]:
%%bigquery
-- LIMIT 0 is a free query; this allows us to check that the table exists.
SELECT * FROM babyweight.babyweight_augmented_data
LIMIT 0

## Lab Task #1: Setup Vertex AI

### Step 1: Open Vertex AI
Go the GCP console and open the console menu in the upper left corner. Then scroll down to the bottom to get to the `Artificial Intelligence` section. Click on `Vertex AI` to open it.

<img src="../assets/vertex/01_vertexai_hamburger_dropdown.png" width='70%'>

### Step 2: Enable API
If you haven't already enabled the Vertex AI API, then you'll see the screen below. Make sure to click the `ENABLE VERTEX AI API` button.

<img src="../assets/vertex/02_vertexai_enable_api.png" width='70%'>

## Lab Task #2: Create and import dataset from BigQuery

### Step 3: Datasets
In the left menu, click `Datasets`.

<img src="../assets/vertex/03_datasets.png" width='70%'>

You should now be on Vertex AI's Datasets page. This is where all imported datasets are shown. We'll want to add our babyweight dataset, so click the `+ CREATE` button.

<img src="../assets/vertex/04_create_dataset.png" width='70%'>

### Step 4: Create new dataset
We need to give our new dataset a unique name. I named mine `babyweight_automl` but feel free to name yours whatever you want.  Select `TABULAR` and `Regression/classification`, as we will build a regression model. When you are done choosing a unique name, click the `CREATE` button.

<img src="../assets/vertex/05_dataset_name.png" width='70%'>

### Step 5: Import your data
Now that we've created a dataset, let's import our data so that Vertex AI and AutoML Tables can use it for training. Our data is currently already in BigQuery, so we will select the radio button `Select a table or view from BigQuery`. This will give a some text box to find our BigQuery table; this field needs the format `project.dataset.table_name`, however, you are able to search for your table just by typing in the table_name, as seen below. Once you are done entering those in, click the `CONTINUE` button.

<img src="../assets/vertex/06_select_bigquery_table.png" width='70%'>

<img src="../assets/vertex/07_selected_bigquery_table.png" width='70%'>

## Lab Task #3: Analyze dataset

### Step 6: Analyze dataset
We can analyze some basic statistics here by clicking `GENERATE STATISTICS`. We can see that we have 6 features, 4 of which are numeric and 2 of which are categorical. We can also see that there are no missing values across all of our columns, which is great! We can also see the number of distinct values which we can compare with our expectations. Once you are satisfied with the analysis, then click the **TRAIN NEW MODEL**. (Note that you can actually start the model training without running statistics, or before the statistics generation is finished, if you are confident enough in your data quality.)

<img src="../assets/vertex/08_generate_statistics.png" width='70%'>

## Lab Task #4: Train AutoML Tables model

### Step 7: Set model type and Select target column
In the section `Training method`, select `Regression` as objective, `AutoML` as model type, and click `Continue`.

<img src="../assets/vertex/09_regression.png" width='70%'>

In the section `Model details`, select `weight_pounds` as the Target Column, and click `Continue`.

<img src="../assets/vertex/10_target_column.png" width='70%'>

### Step 8: Input feature selection
We imported six columns, one of which, `weight_pounds`, we have set aside to be our target or label column. This leaves five columns leftover. The `Training options` dropdown provides you with a list of all of the columns. We want `is_male`, `mother_age`, `plurality`, and `gestation_weeks` as our four features. `hashmonth` is leftover from when we did our repeatable splitting in the [2_prepare_babyweight](../solutions/2_prepare_babyweight.ipynb) lab, that we do not need. Please click the icon on the right to exclude it.

<img src="../assets/vertex/11_feature_selection.png" width='70%'>

### Step 9: Setup training
We are almost ready to train our model. It took a lot of steps to get here but those were mainly to import the data and make sure the data is alright. As we all know, data is extremely important for ML and if it is not what we expect then our model will also not perfom as we expect. Garbage in, garbage out. We need to set the `Training budget` which is the maximum number of node hours to spend training our model. Thankfully, if improvement stops before that, then the training will stop (if the `Enable early stopping` toggle is on) and you'll only be charged for the actual node hours you used. For this dataset, I got decent results with a budget of just 1 node hour. Once that is set the click the `START TRAINING` button.

<img src="../assets/vertex/12_budget.png" width='70%'>

### Step 10: Wait for model to train
Woohoo! Our model is training! We are going to have an awesome model when it finishes! And now we wait. Depending on the size of your dataset, your training budget, and other factors, this could take a while, anywhere from a couple hours to over a day, so this step is about just waiting and being patient. A good thing to do while you are waiting is to keep going through the next labs in this series and then come back to this once lab training completes.

<img src="../assets/vertex/13_automl_tables_training.png" width='70%'>

## Lab Task #5: Check evaluation metrics

### Step 11: Evaluate model
Yay! Our model is done training! If you have received an email that the training job is complete, you can find your way to the model's page either by looking up the Dataset page again, as we saw above (and click on the model name if complete), or in the left sidebar select `Vertex AI`, then `Models`, and look for a recently created model (its `Type` will be `Tabular`).

Now we can check the `EVALUATE` tab and see how well we did. It reminds you what the target was, `weight_pounds`, what the training was optimized for, `RMSE`, and then many evaluation metrics like MAE, RMSE, etc. My training run did great with an RMSE of 1.027 after only an hour of training! It really shows you the amazing power of AutoML! Below you can see a feature importance bar chart. `gestation_weeks` is by far the most important which makes sense because usually the longer someone has been pregnant, the longer the baby has had time to grow, and therefore the heavier the baby weighs.

<img src="../assets/vertex/14_automl_tables_evaluate.png" width='70%'>

## Lab Task #6: Make batch predictions

### Step 12: Create batch prediction job
So if you are satisified with how well our brand new AutoML Tables model trained and evaluated, then you'll probably want to do next what ML is all about; making great predictions! Now that the model is ready, we can create a batch prediction, even without deploying de model for online precitions. To make these easy, we can for now just predict on the BigQuery table that we used to train and evaluate on. To do that, select the tab `BATCH PREDICTIONS`, click `CREATE BATCH PREDICTION`, give any `Batch prediction name`. Select the radio button `BigQuery table` and then enter your `BigQuery Project Id`, `BigQuery Dataset Id`, and `BigQuery Table or View Id`. We could have also used CSVs from Google Cloud Storage. Then we need to select where we want to put our Result. Let's select the Output format `BigQuery` and then enter our `Google Cloud project ID`. We also could have written the results to Google Cloud Storage. Once all of that is set, please click `CREATE` which will submit a batch prediction job using our trained AutoML tabular model and the data at the location we chose above.

<img src="../assets/vertex/15_automl_tables_batch_predict.png" width='70%'>

### Step 13: Batch prediction job finished
After just a little bit of waiting, your batch predictions should be done. For me with my dataset it took less than 30 minutes. Youd should be able to see the finished job in your earlier screen, or navigate from the hamburger menu to `Vertex AI` and `Batch predictions`. Click on your batch prediction name. It shows the data input, where the results are stored, when it was created, and how long it took to process. Click on the `Export location` link to move to the [BigQuery Console UI](https://console.cloud.google.com/bigquery) to have a look.

<img src="../assets/vertex/18_automl_tables_batch_predict_results.png" width='70%'>

### Step 14: Batch prediction dataset
On your list of projects on the far left, you will see the project you have been working in. Click the arrow to expand the dropdown list of all of the BigQuery datasets within the project. You'll see a new dataset there which is the same as what was shown for the `Results directory` from the last step. Expanding that dataset dropdown list you will see two BigQuery tables that have been created: `predictions` and `errors`.

<img src="../assets/vertex/19_automl_tables_batch_predict_dataset.png" width='70%'>

### Step 15: Batch prediction predictions
Let's first look at the `predictions` table, click on the `PREVIEW` tab to view its contents. It has essentially taken your input data to the batch prediction job and appended three new columns to it. Notice even columns that you did not use as features in your model are still here such as `hashmonth`. You should see the two `prediction_inteval` columns for `start` and `end`. The last column is the prediction `value` which for us is our predicted `weight_pounds` that was calculated by our trained AutoML Tables model uses the corresponding features in the row.

<img src="../assets/vertex/20_automl_tables_batch_predict_prediction_table.png" width='70%'>

### Step 16: Batch prediction errors
We can also look at the `errors` table for any possible errors. When I ran my batch prediction job, thankfully I didn't have any errors, but this is definitely the place to check in case you did. Since my `errors` table was empty, below you'll see the schema. Once again it has essentially taken your input data to the batch prediction job and appended three new columns to it. There is a record stored as well as an error `code` and `error` message. These could be helpful in debugging any unwanted behavior.

<img src="../assets/vertex/21_automl_tables_batch_predict_errors_table_schema.png" width='70%'>

## Lab Task #7: Deploy model

### Step 15: Deploy model for online predictions
You might want to also get faster predictions than the above batch job. To do that, we'll have to deploy our trained model. Go to the `DEPLOY & TEST`  tab and then click `DEPLOY TO ENDPOINT`.

<img src="../assets/vertex/16_automl_tables_deploy.png" width='70%'>

### Step 17: Deployment Settings

Give a name to your Endpoint, and you can leave the rest of the settings as default and click `DEPLOY`.

<img src="../assets/vertex/17_deploy_options.png" width='70%'>

## Lab Task #8: Make online predictions

### Step 18: Online prediction setup
Great! Once it is done deploying, you should see an endpoint listed below the `DEPLOY TO ENDPOINT` button, and a `Test your model` area active further down with a table our model's features. Each feature has the column name, the data type, whether it is required or not, and a prepopulated value. You can leave those values as is or enter values. For categorical features, make sure to use valid values or else they will just end up in the OOV (out of vocabulary) spill-over and not take full advantage of the training. When you're done setting your values, click the `PREDICT` button.

<img src="../assets/vertex/22_automl_tables_before_online_predict.png" width='70%'>

### Step 19: Online prediction result
After just a moment, you should see your online predictions appear on your screen. There will be a `Prediction result` as well as a `95% prediction interval` returned. You can try other values for each feature and see what predictions they result in!

<img src="../assets/vertex/23_automl_tables_after_online_predict.png" width='70%'>

## Lab Summary: 
In this lab, we setup AutoML Tables, created and imported an AutoML Tables dataset from BigQuery, analyzed AutoML Tables dataset, trained an AutoML Tables model, checked evaluation metrics of trained model, deployed trained model, and made both batch and online predictions using the trained model.

Copyright 2021 Google LLC
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    https://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.